In [1]:
from __future__ import print_function

# 导入MNIST数据集
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf


/home/hadoop/Gitroot/tensorflow_trial/proj_tf1/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hadoop/Gitroot/tensorflow_trial/proj_tf1/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hadoop/Gitroot/tensorflow_trial/proj_tf1/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0824 00:38:34.206365 139807857805120 deprecation.py:323] From /home/hadoop/Gitroot/tensorflow_trial/proj_tf1/venv/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0824 00:38:34.209413 139807857805120 deprecation.py:323] From /home/hadoop/Gitroot/tensorflow_trial/proj_tf1/venv/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0824 00:38:34.256539 139807857805120 deprecation.py:323] From /home/hadoop/Gitroot/tensorflow_trial/proj_tf1/venv/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/

Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# 超参数
learning_rate = 0.1
num_steps = 10000
batch_size = 128
display_step = 100

# 神经网络参数
n_hidden_1 = 256
n_hidden_2 = 256
num_input = 784
num_classes = 10

# 输入到数据流图中的训练数据
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [3]:
# 权重和偏置
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [4]:
# 定义神经网络
def neural_net(x):
    # 第一层隐藏层（256个神经元）
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # 第二层隐藏层（256个神经元）
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # 输出层
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


In [5]:
# 构建模型
logits = neural_net(X)

# 定义损失函数和优化器
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# 定义预测准确率
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 初始化所有变量（赋默认值）
init = tf.global_variables_initializer()


W0824 00:38:34.498650 139807857805120 deprecation.py:323] From <ipython-input-5-aba26957571c>:6: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
# 开始训练
with tf.Session() as sess:

    # 执行初始化操作
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # 执行训练操作，包括前向和后向传播
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # 计算损失值和准确率
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # 计算测试数据的准确率
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))
    

Step 1, Minibatch Loss= 9491.0645, Training Accuracy= 0.484
Step 100, Minibatch Loss= 357.4123, Training Accuracy= 0.875
Step 200, Minibatch Loss= 119.7758, Training Accuracy= 0.852
Step 300, Minibatch Loss= 160.5068, Training Accuracy= 0.859
Step 400, Minibatch Loss= 74.9545, Training Accuracy= 0.852
Step 500, Minibatch Loss= 39.6674, Training Accuracy= 0.875
Step 600, Minibatch Loss= 20.5948, Training Accuracy= 0.898
Step 700, Minibatch Loss= 46.7842, Training Accuracy= 0.859
Step 800, Minibatch Loss= 35.2611, Training Accuracy= 0.859
Step 900, Minibatch Loss= 59.1861, Training Accuracy= 0.859
Step 1000, Minibatch Loss= 36.4115, Training Accuracy= 0.867
Step 1100, Minibatch Loss= 30.8320, Training Accuracy= 0.922
Step 1200, Minibatch Loss= 70.4084, Training Accuracy= 0.867
Step 1300, Minibatch Loss= 52.4514, Training Accuracy= 0.875
Step 1400, Minibatch Loss= 61.6032, Training Accuracy= 0.867
Step 1500, Minibatch Loss= 54.5858, Training Accuracy= 0.859
Step 1600, Minibatch Loss= 46.2